In [ ]:
from load_preprocess.functions import Data
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [ ]:
data = Data(path = 'data/')
df = data.create_df()

In [ ]:
df_on = pd.read_excel(r'data/road_go_na_ru3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)
df_off = pd.read_excel(r'data/road_go_na_rl3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)

def clean_freight_df(df, on_load=True):
    
    df = df[['TIME', 'TIME.1', '2021']]
    
    if on_load:
        freight_type = "number_onload"
    else:
        freight_type = "number_offload"
        
    df.rename(columns={"TIME": "geo_code", "TIME.1": "geo_labels", "2021": freight_type}, inplace=True)
    df = df.iloc[1:]
    df_clean = df[df.geo_code.str.startswith("FR").fillna(False)]
    df_clean.reset_index(inplace=True, drop=True)
    
    df_clean[freight_type] = df_clean[freight_type].astype('str')
    df_clean.loc[~(df_clean[freight_type].str.isdigit()), freight_type] = '0'
    df_clean[freight_type] = df_clean[freight_type].astype('int')
    
    return df_clean

def merge_freight(df_on, df_off):
    
    df_onload_fr = clean_freight_df(df_on, on_load=True)
    df_offload_fr = clean_freight_df(df_off, on_load=False)

    df_fr = pd.merge(df_onload_fr, df_offload_fr, how='inner', on=['geo_code', 'geo_labels'])
    df_fr['total_load'] = df_fr.number_offload + df_fr.number_onload 
    
    df_fr['geo_labels'] = [c[0] for c in df_fr['geo_labels'].str.split("(")]
    
    return df_fr

df_fr = merge_freight(df_on, df_off)

df_fr